In [ ]:
import pandas as pd
import numpy as np
import random
from deap import base, creator, tools, algorithms
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

def cargar_parametros_desde_excel(ruta_excel='parametros.xlsx'):
    """
    Carga todos los parámetros desde un archivo Excel con múltiples hojas
    
    Args:
        ruta_excel: Ruta al archivo Excel con las hojas de parámetros
        
    Returns:
        params: Diccionario con todos los parámetros cargados
    """
    params = {}
    
    # 1. Cargar parámetros generales desde la hoja "parametros"
    df_params = pd.read_excel(ruta_excel, sheet_name='parametros')
    params['parametros_generales'] = {row['parametro']: row['valor'] for _, row in df_params.iterrows()}
    
    # Convertir tipos de datos según el tipo esperado
    for key in ['almacenamiento', 'costo_plantacion', 'dias_anticipacion', 
                'min_dias_aclimatacion', 'max_dias_aclimatacion', 'dias_simulacion',
                'velocidad_camioneta', 'tiempo_maximo', 'tiempo_carga', 
                'carga_maxima', 'capacidad_maxima_transporte', 'costo_transporte']:
        if key in params['parametros_generales']:
            if key in ['dias_anticipacion', 'min_dias_aclimatacion', 
                      'max_dias_aclimatacion', 'dias_simulacion']:
                params['parametros_generales'][key] = int(params['parametros_generales'][key])
            else:
                params['parametros_generales'][key] = float(params['parametros_generales'][key])
    
    # 2. Cargar hectáreas por polígono desde la hoja "hectareas_poligono"
    df_hectareas = pd.read_excel(ruta_excel, sheet_name='hectareas_poligono')
    params['hectareas_poligono'] = dict(zip(df_hectareas['poligono'], df_hectareas['hectareas']))
    
    # 3. Cargar demanda por especie desde la hoja "demanda_especies"
    df_demanda = pd.read_excel(ruta_excel, sheet_name='demanda_especies')
    params['demanda_especies_por_hectarea'] = dict(zip(df_demanda['especie'], 
                                                     df_demanda['demanda_por_hectarea']))
    
    # 4. Cargar costos de proveedores desde la hoja "costos_proveedores"
    df_costos = pd.read_excel(ruta_excel, sheet_name='costos_proveedores')
    
    # Extraer nombres de proveedores (de las columnas excluyendo la primera)
    params['nombres_proveedores'] = df_costos.columns[1:].tolist()
    
    # Extraer nombres de especies (de la primera columna)
    params['nombres_especies'] = df_costos.iloc[:, 0].tolist()
    
    # Extraer matriz de costos (convertir a float, eliminando la primera columna)
    params['matriz_costos'] = df_costos.iloc[:, 1:].values.astype(float)
    
    # 5. Cargar distancias entre polígonos desde la hoja "distancias_poligonos"
    df_distancias = pd.read_excel(ruta_excel, sheet_name='distancias_poligonos')
    params['distancias_poligonos'] = df_distancias.values
    
    # Información adicional
    print(f"Parámetros cargados correctamente del archivo Excel: {ruta_excel}")
    print(f"Se cargaron {len(params['hectareas_poligono'])} polígonos")
    print(f"Se cargaron {len(params['nombres_especies'])} especies y {len(params['nombres_proveedores'])} proveedores")
    
    return params

def calculate_total_demand(hectareas_poligono, demanda_especies_por_hectarea):
    """Calculate the total demand for each species across all polygons"""
    # Create dictionary to store demand per species per polygon
    polygon_species_demand = {}
    
    # Calculate demand for each polygon based on hectares and species requirements
    for polygon_id, hectares in hectareas_poligono.items():
        polygon_species_demand[polygon_id] = {}
        for species_id, demand_per_hectare in demanda_especies_por_hectarea.items():
            # Calculate demand for this species in this polygon
            polygon_species_demand[polygon_id][species_id] = round(hectares * demand_per_hectare)
    
    # Calculate total demand per species across all polygons
    total_species_demand = {species_id: 0 for species_id in demanda_especies_por_hectarea.keys()}
    for polygon_id, species_demands in polygon_species_demand.items():
        for species_id, demand in species_demands.items():
            total_species_demand[species_id] += demand
    
    return polygon_species_demand, total_species_demand

class InventoryManager:
    """Class to manage plant inventory, acclimation and orders"""
    def __init__(self, demanda_especies_por_hectarea, min_dias_aclimatacion=3, max_dias_aclimatacion=7):
        self.inventory = {}  # {species_id: [(quantity, days_in_inventory), ...]}
        self.pending_orders = {}  # {delivery_day: {species_id: quantity}}
        self.current_day = 0
        self.total_ordered = {species_id: 0 for species_id in demanda_especies_por_hectarea.keys()}
        self.history = []  # Daily inventory snapshots
        self.min_dias_aclimatacion = min_dias_aclimatacion
        self.max_dias_aclimatacion = max_dias_aclimatacion
        
    def place_order(self, orders, delivery_day):
        """Place orders with suppliers for delivery on specified day"""
        if delivery_day not in self.pending_orders:
            self.pending_orders[delivery_day] = {}
        
        for species_id, quantity in orders.items():
            if species_id in self.pending_orders[delivery_day]:
                self.pending_orders[delivery_day][species_id] += quantity
            else:
                self.pending_orders[delivery_day][species_id] = quantity
            
            # Track total ordered per species
            if species_id in self.total_ordered:
                self.total_ordered[species_id] += quantity
    
    def receive_deliveries(self, day):
        """Receive any deliveries scheduled for today"""
        if day in self.pending_orders:
            for species_id, quantity in self.pending_orders[day].items():
                if species_id not in self.inventory:
                    self.inventory[species_id] = []
                # Add to inventory with 0 days acclimation
                self.inventory[species_id].append((quantity, 0))
            # Clear processed orders
            del self.pending_orders[day]
    
    def update_inventory(self):
        """Age inventory by one day and take daily snapshot"""
        # Take snapshot before updating
        snapshot = self._get_inventory_snapshot()
        self.history.append(snapshot)
        
        # Update age of all plants
        for species_id in self.inventory:
            self.inventory[species_id] = [(qty, days + 1) for qty, days in self.inventory[species_id]]
    
    def _get_inventory_snapshot(self):
        """Create a snapshot of current inventory state"""
        snapshot = {
            'total': {},
            'available': {},
            'by_age': {}
        }
        
        for species_id, items in self.inventory.items():
            # Total quantity by species
            total_qty = sum(qty for qty, _ in items)
            snapshot['total'][species_id] = total_qty
            
            # Available quantity (3-7 days)
            avail_qty = sum(qty for qty, days in items if self.min_dias_aclimatacion <= days <= self.max_dias_aclimatacion)
            snapshot['available'][species_id] = avail_qty
            
            # Group by days
            by_age = {}
            for qty, days in items:
                if days not in by_age:
                    by_age[days] = 0
                by_age[days] += qty
            snapshot['by_age'][species_id] = by_age
            
        return snapshot
    
    def get_available_inventory(self):
        """Get inventory items available for transport (3-7 days old)"""
        available = {}
        for species_id, items in self.inventory.items():
            available_qty = sum(qty for qty, days in items if self.min_dias_aclimatacion <= days <= self.max_dias_aclimatacion)
            if available_qty > 0:
                available[species_id] = available_qty
        return available
    
    def get_inventory_summary(self):
        """Get a summary of current inventory"""
        summary = {}
        for species_id, items in self.inventory.items():
            total = sum(qty for qty, _ in items)
            available = sum(qty for qty, days in items if self.min_dias_aclimatacion <= days <= self.max_dias_aclimatacion)
            too_young = sum(qty for qty, days in items if days < self.min_dias_aclimatacion)
            too_old = sum(qty for qty, days in items if days > self.max_dias_aclimatacion)
            
            summary[species_id] = {
                'total': total,
                'available': available,
                'too_young': too_young,
                'too_old': too_old
            }
        return summary
    
    def remove_from_inventory(self, species_distribution):
        """Remove distributed items from inventory, prioritizing oldest items"""
        for species_id, qty_needed in species_distribution.items():
            if species_id not in self.inventory or qty_needed <= 0:
                continue
                
            # Sort by age (oldest first)
            self.inventory[species_id].sort(key=lambda x: x[1], reverse=True)
            
            qty_remaining = qty_needed
            new_inventory = []
            
            for qty, days in self.inventory[species_id]:
                if self.min_dias_aclimatacion <= days <= self.max_dias_aclimatacion and qty_remaining > 0:
                    # This batch is available for distribution
                    if qty <= qty_remaining:
                        # Use entire batch
                        qty_remaining -= qty
                    else:
                        # Use part of batch
                        new_inventory.append((qty - qty_remaining, days))
                        qty_remaining = 0
                else:
                    # Either not available or no more needed
                    new_inventory.append((qty, days))
            
            self.inventory[species_id] = new_inventory

def calculate_route_time(route, dist_poligonos_hrs):
    if len(route) <= 1:
        return 0
    
    total_time = 0
    # Sumar los tiempos de viaje entre polígonos consecutivos
    for i in range(len(route) - 1):
        total_time += dist_poligonos_hrs[route[i], route[i+1]]
    
    # Añadir tiempos de carga/descarga (0.5 hrs por polígono visitado) más 0.5 extra
    # Se resta 2 porque no contamos como paradas el polígono inicial y final si son el mismo
    total_time += 0.5 * (len(route) - 2) + 0.5
    
    return total_time

def generar_ruta_greedy_con_disponibles(poligonos_disponibles, dist_poligonos_hrs, START_POLYGON, tiempo_maximo):
    ruta = [START_POLYGON]  # Comenzar desde el polígono definido
    tiempo_total = 0.0
    poligonos_no_visitados = poligonos_disponibles.copy() 
    
    while poligonos_no_visitados and len(ruta) < 10:  # Limitar a un máximo de 10 polígonos
        # Encontrar el polígono más cercano al último visitado
        actual = ruta[-1]
        nearest = min(poligonos_no_visitados, key=lambda x: dist_poligonos_hrs[actual, x])
        tiempo_a_siguiente = dist_poligonos_hrs[actual, nearest]
        
        # calcular el tiempo total estimado si se añade este polígono
        # Nueva ruta con el polígono más cercano
        new_route = ruta + [nearest]
        new_time = 0
        
        # Sum travel times
        for i in range(len(new_route) - 1):
            new_time += dist_poligonos_hrs[new_route[i], new_route[i+1]]
            
        # Add loading times + extra 0.5
        new_time += 0.5 * (len(new_route) - 1) + 0.5
        
        # Add anticipated time to return to depot
        return_time = dist_poligonos_hrs[nearest, START_POLYGON]
        total_estimated_time = new_time + return_time
        
        # Check if adding this polygon keeps the route under time limit
        if total_estimated_time <= tiempo_maximo:
            ruta.append(nearest)
            poligonos_no_visitados.remove(nearest)
            tiempo_total = new_time  # Update current time without return trip
        else:
            break
    
    # Add return to starting polygon
    ruta.append(START_POLYGON)
    
    # Calculate final time using the calculate_route_time function to ensure consistency
    tiempo_final = calculate_route_time(ruta, dist_poligonos_hrs)
    
    return ruta, tiempo_final

def generar_multiples_rutas_greedy(dist_poligonos_hrs, NUM_POLYGONS, START_POLYGON, tiempo_maximo):
    todas_rutas = []
    poligonos_restantes = [i for i in range(NUM_POLYGONS) if i != START_POLYGON]
    
    while poligonos_restantes:
        # Generate a greedy route with remaining polygons
        ruta, tiempo = generar_ruta_greedy_con_disponibles(
            poligonos_restantes, dist_poligonos_hrs, START_POLYGON, tiempo_maximo
        )
        todas_rutas.append((ruta, tiempo))
        
        # Remove visited polygons from the set of remaining polygons
        for p in ruta[1:-1]:  # Exclude start/end depot
            if p in poligonos_restantes:
                poligonos_restantes.remove(p)
        
        # If we couldn't add any polygons to the route, break to avoid infinite loop
        if len(ruta) <= 2:  # Just start and end nodes
            break
    
    return todas_rutas

def run_simulation(dias_simulacion, dias_anticipacion, min_dias_aclimatacion, max_dias_aclimatacion,
                  hectareas_poligono, demanda_especies_por_hectarea, capacidad_maxima_transporte,
                  costo_transporte, carga_maxima, nombres_especies, nombres_proveedores, 
                  matriz_costos, dist_poligonos_hrs, NUM_POLYGONS, START_POLYGON):
    """Run the full simulation for the specified number of days"""
    # Calculate initial demands
    polygon_species_demand, total_species_demand = calculate_total_demand(
        hectareas_poligono=hectareas_poligono, 
        demanda_especies_por_hectarea=demanda_especies_por_hectarea
    )
    
    # Convert total demand to list format for reporting
    initial_demand_list = [total_species_demand.get(i, 0) for i in range(1, len(nombres_especies) + 1)]
    
    print(f"\n--- DEMANDA INICIAL ---")
    for i, demand in enumerate(initial_demand_list):
        print(f"{nombres_especies[i]}: {demand}")
    
    # Create inventory manager
    inventory = InventoryManager(
        demanda_especies_por_hectarea,
        min_dias_aclimatacion=min_dias_aclimatacion,
        max_dias_aclimatacion=max_dias_aclimatacion
    )
    
    # Daily tracking of orders, routes, and demand fulfillment
    daily_orders = {}
    daily_routes = {}
    daily_distributions = {}
    daily_demand_coverage = []
    
    # Deep copy of initial demand for tracking
    current_demand = {polygon: {species: qty for species, qty in species_demand.items()} 
                    for polygon, species_demand in polygon_species_demand.items()}
    
    for day in range(dias_simulacion):
        print(f"\n{'='*50}")
        print(f"SIMULACIÓN DÍA {day+1}")
        print(f"{'='*50}")
        
        # 1. Receive any pending deliveries
        inventory.receive_deliveries(day)
        
        # 2. Generate routes for today
        daily_routes[day] = generar_multiples_rutas_greedy(
            dist_poligonos_hrs=dist_poligonos_hrs,
            NUM_POLYGONS=NUM_POLYGONS,
            START_POLYGON=START_POLYGON,
            tiempo_maximo=tiempo_maximo
        )
        
        # 3. Distribute available plants to routes
        distribution_plan, route_loads = distribute_plants_to_routes(
            inventory, current_demand, daily_routes[day], carga_maxima
        )
        daily_distributions[day] = distribution_plan
        
        # 4. Update remaining demand
        current_demand = update_demand_after_distribution(
            current_demand, distribution_plan
        )
        
        # 5. Calculate demand coverage
        coverage = calculate_demand_coverage(total_species_demand, current_demand)
        daily_demand_coverage.append(coverage)
        
        # 6. Order plants for future delivery (simple ordering strategy)
        current_orders = None
        if day < dias_simulacion - dias_anticipacion:
            # Order for delivery in dias_anticipacion days
            delivery_day = day + dias_anticipacion

            # Calculate remaining demand across all polygons
            remaining_demand = {}
            for polygon, species_demands in current_demand.items():
                for species_id, demand in species_demands.items():
                    if demand > 0:
                        if species_id in remaining_demand:
                            remaining_demand[species_id] += demand
                        else:
                            remaining_demand[species_id] = demand
            
            # Place orders with suppliers
            if remaining_demand:
                current_orders = place_orders_with_suppliers(
                    inventory, remaining_demand, matriz_costos, costo_transporte, 
                    capacidad_maxima_transporte, day, delivery_day, 
                    nombres_especies, nombres_proveedores
                )
                daily_orders[day] = (delivery_day, current_orders)
        
        # 7. Generate daily report
        generar_reporte_diario(
            day, current_orders, daily_routes[day], distribution_plan, route_loads,
            inventory, current_demand, coverage, nombres_especies
        )
        
        # 8. Update inventory ages
        inventory.update_inventory()

        # 9. Visualize route distributions
        visualizar_distribucion_rutas(
            daily_routes[day], distribution_plan, day, nombres_especies,
            NUM_POLYGONS, START_POLYGON, dist_poligonos_hrs
        )
    
    # End of simulation visualizations
    print("\n--- RESUMEN FINAL DE SIMULACIÓN ---")
    print(f"Días simulados: {dias_simulacion}")
    
    # Visualize demand coverage over time
    visualizar_cobertura_demanda(daily_demand_coverage, nombres_especies)
    
    # Visualize inventory history
    visualizar_inventario(inventory.history, nombres_especies)
    
    # Return simulation data
    return {
        "daily_orders": daily_orders,
        "daily_routes": daily_routes,
        "daily_distributions": daily_distributions,
        "daily_demand_coverage": daily_demand_coverage,
        "final_demand": current_demand,
        "inventory_history": inventory.history
    }

# Funciones complementarias que necesitan ser modificadas para recibir parámetros en lugar de usar variables globales
def visualizar_distribucion_rutas(routes, distribution_plan, day, nombres_especies, NUM_POLYGONS, START_POLYGON, dist_poligonos_hrs):
    """Visualize route distributions"""
    if not routes:
        return
    
    num_routes = min(4, len(routes))
    plt.figure(figsize=(16, 12))
    
    for i in range(num_routes):
        plt.subplot(2, 2, i+1)
        route, time = routes[i]
        
        # Create graph
        G = nx.DiGraph()
        
        # Add nodes
        for j in range(NUM_POLYGONS):
            G.add_node(j)
        
        # Add edges in the route
        for j in range(len(route) - 1):
            G.add_edge(route[j], route[j+1], weight=dist_poligonos_hrs[route[j], route[j+1]])
        
        pos = nx.spring_layout(G, seed=42)
        
        # Draw basic graph
        nx.draw(G, pos, with_labels=True, node_color='lightblue', 
                node_size=500, arrowsize=20, font_weight='bold')
        
        # Highlight the depot
        nx.draw_networkx_nodes(G, pos, nodelist=[START_POLYGON], node_color='red', node_size=700)
        
        # Add distribution info to title
        if i in distribution_plan:
            total_plants = sum(sum(species_dict.values()) 
                              for polygon_dict in distribution_plan[i].values() 
                              for species_dict in [polygon_dict])
            plt.title(f"Día {day+1}, Ruta {i+1}: Tiempo = {time:.2f} hrs, {total_plants} plantas")
        else:
            plt.title(f"Día {day+1}, Ruta {i+1}: Tiempo = {time:.2f} hrs")
            
        plt.axis('off')
    
    plt.suptitle(f"Distribución de Rutas - Día {day+1}", fontsize=16)
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.show()

# Aquí mantendría el resto de funciones como place_orders_with_suppliers, distribute_plants_to_routes, etc.
# que no modifiqué para mantener el código breve, pero considera que también deben adaptarse para usar
# los parámetros recibidos en lugar de variables globales.

# Función principal
def main():
    # Cargar todos los parámetros desde archivo Excel
    params = cargar_parametros_desde_excel('parametros.xlsx')
    
    # Extraer parámetros generales
    parametros_generales = params['parametros_generales']
    almacenamiento = parametros_generales.get('almacenamiento', 400)
    costo_plantacion = parametros_generales.get('costo_plantacion', 20)
    dias_anticipacion = parametros_generales.get('dias_anticipacion', 1)
    min_dias_aclimatacion = parametros_generales.get('min_dias_aclimatacion', 3)
    max_dias_aclimatacion = parametros_generales.get('max_dias_aclimatacion', 7)
    dias_simulacion = parametros_generales.get('dias_simulacion', 30)
    velocidad_camioneta = parametros_generales.get('velocidad_camioneta', 40)
    tiempo_maximo = parametros_generales.get('tiempo_maximo', 6)
    tiempo_carga = parametros_generales.get('tiempo_carga', 0.5)
    carga_maxima = parametros_generales.get('carga_maxima', 524)
    capacidad_maxima_transporte = parametros_generales.get('capacidad_maxima_transporte', 8000)
    costo_transporte = parametros_generales.get('costo_transporte', 4500)
    start_polygon = int(parametros_generales.get('start_polygon', 6))
    
    # Extraer otros datos
    hectareas_poligono = params['hectareas_poligono']
    demanda_especies_por_hectarea = params['demanda_especies_por_hectarea']
    nombres_especies = params['nombres_especies']
    nombres_proveedores = params['nombres_proveedores']
    matriz_costos = params['matriz_costos']
    
    # Procesar matriz de distancias
    distancias_poligonos = params['distancias_poligonos']
    dist_poligonos_hrs = distancias_poligonos / velocidad_camioneta
    dist_poligonos_hrs = np.round(dist_poligonos_hrs, 2)
    
    # Determinar el número total de polígonos
    NUM_POLYGONS = max(hectareas_poligono.keys()) + 1  # +1 para incluir todos los polígonos
    START_POLYGON = start_polygon
    
    # Imprimir parámetros cargados
    print("\n--- PARÁMETROS DE SIMULACIÓN (CARGADOS DE EXCEL) ---")
    print(f"Días de simulación: {dias_simulacion}")
    print(f"Capacidad máxima de transporte: {capacidad_maxima_transporte}")
    print(f"Días de anticipación para pedidos: {dias_anticipacion}")
    print(f"Días mínimos de aclimatación: {min_dias_aclimatacion}")
    print(f"Días máximos de aclimatación: {max_dias_aclimatacion}")
    print(f"Capacidad de carga por ruta: {carga_maxima}")
    print(f"Tiempo máximo por ruta: {tiempo_maximo} horas")
    
    # Ejecutar simulación con los parámetros cargados
    simulation_results = run_simulation(
        dias_simulacion=dias_simulacion,
        dias_anticipacion=dias_anticipacion,
        min_dias_aclimatacion=min_dias_aclimatacion,
        max_dias_aclimatacion=max_dias_aclimatacion,
        hectareas_poligono=hectareas_poligono,
        demanda_especies_por_hectarea=demanda_especies_por_hectarea,
        capacidad_maxima_transporte=capacidad_maxima_transporte,
        costo_transporte=costo_transporte,
        carga_maxima=carga_maxima,
        nombres_especies=nombres_especies,
        nombres_proveedores=nombres_proveedores,
        matriz_costos=matriz_costos,
        dist_poligonos_hrs=dist_poligonos_hrs,
        NUM_POLYGONS=NUM_POLYGONS,
        START_POLYGON=START_POLYGON
    )
    
    # Final summary messages
    final_coverage = simulation_results["daily_demand_coverage"][-1]
    avg_coverage = sum(final_coverage.values()) / len(final_coverage)
    
    print("\n--- RESULTADO FINAL ---")
    print(f"Cobertura de demanda promedio: {avg_coverage:.2%}")
    
    total_orders = sum(
        sum(providers.values()) for day_info in simulation_results["daily_orders"].values()
        for _, orders in [day_info] for species, providers in orders.items()
    )
    
    print(f"Total de plantas ordenadas: {total_orders}")
    print("Simulación completada con éxito!")

if __name__ == "__main__":
    main()